In [ ]:
# 1.1 Instalación de librerías
#pip install llama-index qdrant-client llama-index-vector-stores-qdrant llama-index-llms-ollama llama-index-embeddings-ollama pypdf ollama Ipython
# pyhton 3.10
# 1.2 Importaciones necesarias
import json
import qdrant_client
from collections import Counter
from llama_index.core import Document, VectorStoreIndex, Settings, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
import logging
import sys

# (Opcional) Configurar logging para ver más detalles del proceso
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# 1.3 Configuración de los modelos y servicios (usando tu configuración previa)
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
Settings.llm = Ollama(model="deepseek-r1:14b", base_url="https://ollama01.decoupled.ai", request_timeout=120)

# Conexión a Qdrant
client = qdrant_client.QdrantClient(
    url='https://qdrant01.decoupled.ai:443',
    timeout=60,
    api_key=None, # O tu API key si la tienes
)

collection_name = "exist2025_task1_2_rag"
vector_store = QdrantVectorStore(client=client, collection_name=collection_name)

In [ ]:
# 1.1 Instalación de librerías
#enviroment python 3.10
#pip install llama-index qdrant-client llama-index-vector-stores-qdrant llama-index-llms-ollama llama-index-embeddings-ollama pypdf ollama
# 1.2 Importaciones necesarias
import sys
import logging
import qdrant_client

from IPython.display import Markdown, display
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

# (Opcional) Configurar logging para ver más detalles del proceso
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#documents = SimpleDirectoryReader("./data_pdf/").load_data()


client = qdrant_client.QdrantClient(
    url='https://qdrant01.decoupled.ai:443',
    timeout=60,
    api_key=None,
)

ollama_embedding = OllamaEmbedding(
    model_name="mxbai-embed-large",
    base_url="https://ollama03.decoupled.ai",
    ollama_additional_kwargs={"mirostat": 0},
)

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-base-en-v1.5")
Settings.embed_model = ollama_embedding
Settings.llm = Ollama(model="qwen2.5:latest", base_url="https://ollama01.decoupled.ai", request_timeout=60)

collection_name = "exist2025_task1_2_rag"
vector_store = QdrantVectorStore(client=client, collection_name=collection_name)



#py -3.10 -m pvenv venv_py311  activarenv  pip install llama-index qdrant-client llama-index-vector-stores-qdrant llama-index-llms-ollama llama-index-embeddings-ollama pypdf ollama

INFO:httpx:HTTP Request: GET https://qdrant01.decoupled.ai "HTTP/1.1 200 OK"
HTTP Request: GET https://qdrant01.decoupled.ai "HTTP/1.1 200 OK"
HTTP Request: GET https://qdrant01.decoupled.ai "HTTP/1.1 200 OK"


C:\Users\luisa\AppData\Local\Temp\ipykernel_31296\3585774019.py:24: UserWarning: Qdrant client version 1.15.1 is incompatible with server version 1.10.0. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  client = qdrant_client.QdrantClient(


INFO:httpx:HTTP Request: GET https://qdrant01.decoupled.ai/collections/exist2025_task1_2_rag/exists "HTTP/1.1 200 OK"
HTTP Request: GET https://qdrant01.decoupled.ai/collections/exist2025_task1_2_rag/exists "HTTP/1.1 200 OK"
HTTP Request: GET https://qdrant01.decoupled.ai/collections/exist2025_task1_2_rag/exists "HTTP/1.1 200 OK"


In [10]:
import json
from collections import Counter  
from llama_index.core import Document
# 2.1 Función para procesar cada tweet del JSON
def process_tweet_entry(tweet_data):
    """
    Toma un objeto de tweet del dataset y lo convierte en un
    Document de LlamaIndex con metadatos procesados para la Tarea 1.2.
    """
    # Filtra las etiquetas de la Tarea 1.2, ignorando los tuits no-sexistas ('-')
    valid_labels = [label for label in tweet_data['labels_task1_2'] if label != "-"]
    
    # Si no hay etiquetas válidas, significa que la mayoría votó NO en la Tarea 1.1.
    # No incluimos este ejemplo en nuestra base de conocimiento para la Tarea 1.2.
    if not valid_labels:
        return None

    # Calcula el voto mayoritario para la etiqueta 'hard'
    vote_counts = Counter(valid_labels)
    majority_label = vote_counts.most_common(1)[0][0]

    # Calcula la distribución 'soft'
    total_valid_votes = len(valid_labels)
    soft_label = {label: count / total_valid_votes for label, count in vote_counts.items()}

    # Construye el diccionario de metadatos
    metadata = {
        "id_EXIST": tweet_data['id_EXIST'],
        "lang": tweet_data['lang'],
        "majority_label": majority_label,
        "soft_label_str": json.dumps(soft_label) # Guardamos el dict como string
    }
    
    # Crea y devuelve el Documento
    return Document(text=tweet_data['tweet'], metadata=metadata)

# 2.2 Cargar el fichero JSON y procesar los datos de entrenamiento
file_path = '../data/EXIST2025_training.json' # <-- CAMBIA ESTA RUTA
documents_for_rag = []

print("Cargando y procesando el dataset...")
with open(file_path, 'r', encoding='utf-8') as f:
    full_dataset = json.load(f)

# El dataset es un diccionario, así que iteramos sobre sus valores
for tweet_obj in full_dataset.values():
    # Solo procesamos los datos del conjunto de entrenamiento
    if 'TRAIN' in tweet_obj['split']:
        doc = process_tweet_entry(tweet_obj)
        if doc:
            documents_for_rag.append(doc)

print(f"Procesamiento completo. Se han creado {len(documents_for_rag)} documentos para el RAG.")
# Opcional: Imprime un ejemplo para verificar
if documents_for_rag:
    print("\nEjemplo de un documento procesado:")
    print(f"Texto: {documents_for_rag[0].text[:100]}...")
    print(f"Metadatos: {documents_for_rag[0].metadata}")

Cargando y procesando el dataset...
Procesamiento completo. Se han creado 5486 documentos para el RAG.

Ejemplo de un documento procesado:
Texto: @TheChiflis Ignora al otro, es un capullo.El problema con este youtuber denuncia el acoso... cuando ...
Metadatos: {'id_EXIST': '100001', 'lang': 'es', 'majority_label': 'REPORTED', 'soft_label_str': '{"REPORTED": 0.6, "JUDGEMENTAL": 0.4}'}


In [11]:
# 3.1 Crear el contexto de almacenamiento y el índice
print(f"Indexando {len(documents_for_rag)} documentos en la colección '{collection_name}' de Qdrant...")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents_for_rag,
    storage_context=storage_context,
    show_progress=True
)
print("¡Indexación completada con éxito!")

Indexando 5486 documentos en la colección 'exist2025_task1_2_rag' de Qdrant...


Parsing nodes: 100%|██████████| 5486/5486 [00:01<00:00, 3222.42it/s]


INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST ht

INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST ht

INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST ht

...

INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST ht

INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST ht

INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST ht

Generating embeddings: 100%|██████████| 1390/1390 [03:27<00:00,  6.68it/s]


INFO:httpx:HTTP Request: PUT https://qdrant01.decoupled.ai/collections/exist2025_task1_2_rag/points?wait=true "HTTP/1.1 200 OK"
HTTP Request: PUT https://qdrant01.decoupled.ai/collections/exist2025_task1_2_rag/points?wait=true "HTTP/1.1 200 OK"
HTTP Request: PUT https://qdrant01.decoupled.ai/collections/exist2025_task1_2_rag/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT https://qdrant01.decoupled.ai/collections/exist2025_task1_2_rag/points?wait=true "HTTP/1.1 200 OK"
HTTP Request: PUT https://qdrant01.decoupled.ai/collections/exist2025_task1_2_rag/points?wait=true "HTTP/1.1 200 OK"
HTTP Request: PUT https://qdrant01.decoupled.ai/collections/exist2025_task1_2_rag/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT https://qdrant01.decoupled.ai/collections/exist2025_task1_2_rag/points?wait=true "HTTP/1.1 200 OK"
HTTP Request: PUT https://qdrant01.decoupled.ai/collections/exist2025_task1_2_rag/points?wait=true "HTTP/1.1 200 OK"
HTTP Request: PUT https://qdran

In [14]:
# 4.1 Cargar el índice desde Qdrant (así no tienes que re-indexar cada vez)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

# 4.2 Crear un retriever para buscar los 3 ejemplos más similares
retriever = index.as_retriever(similarity_top_k=3)

# 4.3 ¡Hagamos una prueba!
new_post_to_classify = "Las mujeres solo sirven para estar en la cocina, no para la política."
print(f"Buscando ejemplos para el post: '{new_post_to_classify}'\n")

retrieved_nodes = retriever.retrieve(new_post_to_classify)

# 4.4 Construir el prompt dinámicamente con los resultados
examples_string = ""
print("--- Ejemplos Recuperados ---")
for i, node in enumerate(retrieved_nodes):
    examples_string += f"--- \n"
    examples_string += f"Ejemplo {i+1}: {node.get_content()} \n"
    examples_string += f"Intención: {node.metadata['majority_label']} \n" # Usamos la etiqueta de voto mayoritario
    print(f"Ejemplo {i+1} (Similitud: {node.get_score():.4f}):")
    print(f"  Texto: {node.get_content()}")
    print(f"  Intención (etiqueta mayoritaria): {node.metadata['majority_label']}")

# 4.5 Crear el prompt final para el LLM
final_prompt = f"""
Analiza la intención del siguiente 'Post'. Las posibles intenciones son DIRECT, REPORTED, o JUDGEMENTAL.
Aquí tienes algunos ejemplos para guiarte:
{examples_string}
---
Ahora, clasifica el siguiente post:
Post: "{new_post_to_classify}"
Intención:
"""

print("\n\n--- PROMPT FINAL GENERADO ---")
print(final_prompt)

#(Opcional) Enviar al LLM para obtener la respuesta final
response = Settings.llm.complete(final_prompt)
print("\n--- RESPUESTA DEL LLM ---")
print(response)

Buscando ejemplos para el post: 'Las mujeres solo sirven para estar en la cocina, no para la política.'

INFO:httpx:HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://ollama03.decoupled.ai/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://qdrant01.decoupled.ai/collections/exist2025_task1_2_rag/points/search "HTTP/1.1 200 OK"
HTTP Request: POST https://qdrant01.decoupled.ai/collections/exist2025_task1_2_rag/points/search "HTTP/1.1 200 OK"
HTTP Request: POST https://qdrant01.decoupled.ai/collections/exist2025_task1_2_rag/points/search "HTTP/1.1 200 OK"
--- Ejemplos Recuperados ---
Ejemplo 1 (Similitud: 0.6490):
  Texto: @psalaburu @PedroUgarte6 Ahí tienes un montón de sociedades gastronómicas en las que no se les permite la entrada a la cocina a mujeres, o en las que la mujeres no pueden ser socias. Puedes abrir ese melón también, si quieres.
 

ModuleNotFoundError: No module named 'llama_index'